# ***Movie Recommendation System***

In [9]:
import pandas as pd
import numpy as np  
import matplotlib
import matplotlib.pyplot as plt  
%matplotlib inline
import seaborn as sns

In [22]:
from google.colab import files
uploaded = files.upload()

Saving tmdb_5000_credits.csv to tmdb_5000_credits (2).csv


In [23]:
import io
credits = pd.read_csv(io.BytesIO(uploaded['tmdb_5000_credits.csv']))
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [25]:
movies = pd.read_csv('tmdb_5000_movies.csv')
movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [26]:
 credits.shape

(4803, 4)

In [27]:
movies.shape

(4803, 20)

In [29]:
credits_column_renamed=credits.rename(index=str,columns={"movie_id":"id"})
movies_merge=movies.merge(credits_column_renamed,on="id")
movies_merge.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [30]:
movies_merge=movies_merge.drop(columns=['homepage','title_x','title_y','status','production_countries','production_companies','revenue','vote_average','vote_count'])

In [31]:
movies_merge.head()

,budget,genres,id,keywords,original_language,original_title,overview,popularity,release_date,runtime,spoken_languages,tagline,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Enter the World of Pandora.,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","At the end of the world, the adventure begins.","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",A Plan No One Escapes,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,2012-07-16,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",The Legend Ends,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,2012-03-07,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","Lost in our world, found in another.","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [32]:
movies_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             4803 non-null   int64  
 1   genres             4803 non-null   object 
 2   id                 4803 non-null   int64  
 3   keywords           4803 non-null   object 
 4   original_language  4803 non-null   object 
 5   original_title     4803 non-null   object 
 6   overview           4800 non-null   object 
 7   popularity         4803 non-null   float64
 8   release_date       4802 non-null   object 
 9   runtime            4801 non-null   float64
 10  spoken_languages   4803 non-null   object 
 11  tagline            3959 non-null   object 
 12  cast               4803 non-null   object 
 13  crew               4803 non-null   object 
dtypes: float64(2), int64(2), object(10)
memory usage: 562.9+ KB


In [34]:
movies_merge['overview'].head(2)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
Name: overview, dtype: object

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv= TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode',
                     analyzer='word',token_pattern=r'\w{1,}', stop_words='english')

movies_merge['overview']=movies_merge['overview'].fillna('')

In [38]:
tfv_matrix=tfv.fit_transform(movies_merge['overview'])

In [39]:
tfv_matrix

<4803x7591 sparse matrix of type '<class 'numpy.float64'>'
	with 112085 stored elements in Compressed Sparse Row format>

In [40]:
tfv_matrix.shape

(4803, 7591)

In [42]:
from sklearn.metrics.pairwise import sigmoid_kernel

sig=sigmoid_kernel(tfv_matrix,tfv_matrix)

In [43]:
sig[0:5]

array([[0.76164948, 0.76159416, 0.76159416, ..., 0.76159416, 0.76159416,
        0.76159416],
       [0.76159416, 0.76164948, 0.76159416, ..., 0.76159565, 0.76159416,
        0.76159416],
       [0.76159416, 0.76159416, 0.76164948, ..., 0.76159524, 0.76159416,
        0.76159481],
       [0.76159566, 0.76159416, 0.76159459, ..., 0.7615965 , 0.76159679,
        0.76159676],
       [0.76159416, 0.76159668, 0.76159457, ..., 0.76159487, 0.76159416,
        0.76159536]])

In [45]:
indices=pd.Series(movies_merge.index,index=movies_merge['original_title']).drop_duplicates()


In [46]:
indices

original_title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [48]:
indices['The Dark Knight Rises']

3

In [49]:
sig[3]

array([0.76159566, 0.76159416, 0.76159459, ..., 0.7615965 , 0.76159679,
       0.76159676])

In [50]:
list(enumerate(sig[indices['The Dark Knight Rises']]))

[(0, 0.7615956631855698),
 (1, 0.7615941559557649),
 (2, 0.7615945874962927),
 (3, 0.7616494757038861),
 (4, 0.7615956339980098),
 (5, 0.761594830785211),
 (6, 0.7615961738189113),
 (7, 0.7615963246930962),
 (8, 0.7615958145222526),
 (9, 0.7616035745614476),
 (10, 0.7615941559557649),
 (11, 0.7615941559557649),
 (12, 0.7615941559557649),
 (13, 0.7615941559557649),
 (14, 0.7615941559557649),
 (15, 0.7615944626600168),
 (16, 0.7615957350540753),
 (17, 0.7615959261414685),
 (18, 0.7615946832656142),
 (19, 0.76159553011228),
 (20, 0.7615948946225571),
 (21, 0.7615978178784655),
 (22, 0.7615941559557649),
 (23, 0.7615946414135039),
 (24, 0.7615952957490761),
 (25, 0.7615954659528092),
 (26, 0.7615953447314181),
 (27, 0.7615947351144076),
 (28, 0.7615948303452853),
 (29, 0.7615959134493275),
 (30, 0.7615955957562675),
 (31, 0.7615967400590945),
 (32, 0.7615956024391881),
 (33, 0.7615946642331725),
 (34, 0.7615941559557649),
 (35, 0.7615941559557649),
 (36, 0.7615960891271181),
 (37, 0.761594

In [55]:
sorted(list(enumerate(sig[indices['The Dark Knight Rises']])), key=lambda x: x[1],reverse=True)

[(3, 0.7616494757038861),
 (299, 0.7616144800923044),
 (65, 0.7616115932795493),
 (1359, 0.7616113399737864),
 (428, 0.761610881035617),
 (2507, 0.7616048873438304),
 (3854, 0.761604667777608),
 (119, 0.7616036784667298),
 (9, 0.7616035745614476),
 (1181, 0.7616030786320236),
 (210, 0.7616021321175495),
 (879, 0.7616015493965745),
 (1068, 0.7616002884230638),
 (2416, 0.7616002195900535),
 (1398, 0.7616001547035285),
 (286, 0.7616001104063466),
 (1143, 0.7616000855052019),
 (1202, 0.7615999320273141),
 (2193, 0.7615999179541562),
 (1766, 0.7615995615413677),
 (979, 0.761599439050146),
 (1984, 0.7615993249020594),
 (777, 0.7615992550325601),
 (3819, 0.7615992455202197),
 (790, 0.7615991948949508),
 (2638, 0.7615989746152214),
 (1349, 0.761598863876873),
 (614, 0.761598778905408),
 (1230, 0.7615986903795102),
 (1322, 0.7615986738178375),
 (1492, 0.7615986646847058),
 (4699, 0.7615985940460395),
 (2035, 0.7615985928201457),
 (672, 0.7615985694655207),
 (3914, 0.7615985053708165),
 (2274, 0

In [63]:
def recommendation(title,sig=sig):

  index=indices[title]

  sig_score=list(enumerate(sig[index]))

  sig_sorted=sorted(list(enumerate(sig[index])),key= lambda x:x[1],reverse=True)

  sig_sorted=sig_sorted[1:20]

  movies_recommended= [i[0] for i in sig_sorted]

  return movies_merge['original_title'].iloc[movies_recommended]

In [64]:
#testing
recommendation('Avatar')

3604                          Apollo 18
2130                       The American
1341                   Obitaemyy Ostrov
634                          The Matrix
529                    Tears of the Sun
311        The Adventures of Pluto Nash
151                             Beowulf
847                            Semi-Pro
2628                Blood and Chocolate
1610                              Hanna
570                              Ransom
942                    The Book of Life
775                           Supernova
36      Transformers: Age of Extinction
1033                           Insomnia
1784                          EverAfter
3070            Jeff, Who Lives at Home
3724                      Falcon Rising
1013                           Child 44
Name: original_title, dtype: object